In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from backend.data_processing_service import DataProcessingService
from backend.model_service import ModelService, LinRegNN

# Initialize DataProcessingService
data_processor = DataProcessingService(seq_length=10, pred_window=1, batch_size=10)
data = np.loadtxt('data/normalized_apple_prices.csv')
# data = data_processor.get_sample_data(length=50)

# Normalize the data
data_normalized, scaler = data_processor.normalize_data(data)

# Create sequences
x_data, y_data = data_processor.create_sequences(data_normalized)

# Split data into training and test sets
x_train, x_test, y_train, y_test = data_processor.split_data(x_data, y_data, train_ratio=0.8)

# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [2]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

torch.Size([102, 10, 1]) torch.Size([102, 1, 1]) torch.Size([26, 10, 1]) torch.Size([26, 1, 1])


In [ ]:
# Define the search space for hyperparameters
model_params_space = {
    'input_size': data_processor.seq_length,
    'output_size': data_processor.pred_window
}

# Run hyperparameter optimization
best_hyperparams = ModelService.hyperparameter_optimization(LinRegNN, x_train, y_train, x_test, y_test, model_params_space, max_evals=5)

In [10]:
# Extract best hyperparameters
# best_hidden_size = best_hyperparams['model_params']['hidden_size']
best_learning_rate = best_hyperparams['learning_rate']
best_epochs = best_hyperparams['epochs']
best_hyperparams

{'epochs': 9.0, 'learning_rate': 0.010055235826599312}

In [6]:
# Initialize the model with best hyperparameters
model = LinRegNN(data_processor.seq_length, data_processor.pred_window)

In [8]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
# x_train_rs,x_test_rs= model.reshape_input(x_train),model.reshape_input(x_test)
epochs = 10
model.train_loop(model, criterion, optimizer, x_train, y_train, epochs, directory = "backend/checkpoints/lr01_")
# Eval
model.evaluation(model, x_test, y_test, criterion)

Epoch [1/10], Loss: 0.1209
Epoch [2/10], Loss: 0.1011
Epoch [3/10], Loss: 0.0839
Epoch [4/10], Loss: 0.0697
Epoch [5/10], Loss: 0.0582
Checkpoint saved. Epoch 5
Checkpoint saved at backend/checkpoints/lr01_5.pth
Epoch [6/10], Loss: 0.0492
Epoch [7/10], Loss: 0.0421
Epoch [8/10], Loss: 0.0367
Epoch [9/10], Loss: 0.0325
Epoch [10/10], Loss: 0.0294
Checkpoint saved. Epoch 10
Checkpoint saved at backend/checkpoints/lr01_10.pth
Test Loss: 0.0158


0.015752632259774985

In [ ]:
model.register(model, name="template_test")

Entire model saved successfully.


In [ ]:
from backend.model_service import ModelService
model_service = ModelService()
loaded_model = model_service.load_registered_model("backend/models/template_test")

Entire model loaded successfully.
